In [1]:
from p_ms4_geoff import sort_dataset
import os
import numpy as np

In [2]:
def find_sub(string, sub):
    '''finds all instances of a substring within a string and outputs a list of indices'''
    result = []
    k = 0
    while k < len(string):
        k = string.find(sub, k)
        if k == -1:
            return result
        else:
            result.append(k)
            k += 1  # change to k += len(sub) to not search overlapping results
    return result


def get_ubuntu_path(filepath):
    # get the drive letter

    drive_letter_i = filepath.find(':/')

    if drive_letter_i == -1:
        drive_letter_i = filepath.find(':\\')

    drive_letter = filepath[:drive_letter_i].lower()

    i = 1
    while drive_letter_i + i == '/':
        i += 1

    remaining_path = filepath[drive_letter_i + i + 1:]
    linux_path = '/mnt/%s/%s' % (drive_letter, remaining_path)
    
    # add single quotes so linux can understand the special characters
    if '(' in linux_path or ')' in linux_path:
        linux_path = "'%s'" % linux_path

    return os.path.normpath((linux_path)).replace('\\', '/')

def get_windows_filename(filename):
    # remove the single quotes if added
    if filename[0] == "'" and filename[-1] == "'":
        filename = filename[1:-1]

    filename_split = filename.split('/')
    mnt_i = np.where(np.array(filename_split) == 'mnt')[0][0]

    drive_letter = filename_split[mnt_i + 1].upper()

    remaining = '\\'.join(list(filename_split[mnt_i + 2:]))

    return '%s:\\%s' % (drive_letter, remaining)

### Note I use get_ubuntu_path because I use the Windows Subsystem for Linux so I convert my windows paths to what they get mapped to in Linux. If you are on a Linux machine you can skip the get_ubunut_path parts.

In [17]:
directory = get_ubuntu_path('E:\\Apollo_D_Drive\\data\\VirtualMazeData\\b6_march_18_1')
directory = get_ubuntu_path('E:\\Apollo_D_Drive\\data\VirtualMazeData\\b6_august_18_2\\NoiseTesting')
# directory = get_ubuntu_path('E:\\Apollo_D_Drive\\data\\VirtualMazeData\\b6_august_18_2\\SimpleCircularTrack')

raw_fnames = [os.path.join(directory, file) for file in os.listdir(directory) if '_raw.mda' in file]

for file in raw_fnames:
    print(file)

/mnt/e/Apollo_D_Drive/data/VirtualMazeData/b6_august_18_2/NoiseTesting/b6_august_18_2_in_cage_180907_141509_T1_raw.mda
/mnt/e/Apollo_D_Drive/data/VirtualMazeData/b6_august_18_2/NoiseTesting/b6_august_18_2_in_cage_180907_141509_T2_raw.mda
/mnt/e/Apollo_D_Drive/data/VirtualMazeData/b6_august_18_2/NoiseTesting/b6_august_18_2_in_cage_180907_141509_T3_raw.mda
/mnt/e/Apollo_D_Drive/data/VirtualMazeData/b6_august_18_2/NoiseTesting/b6_august_18_2_in_cage_180907_141509_T4_raw.mda


In [18]:
raw_fname = get_windows_filename(raw_fnames[0])

print(raw_fname)

mda_basename = os.path.splitext(raw_fname)[0]
mda_basename = mda_basename[:find_sub(mda_basename, '_')[-1]]

masked_out_fname = get_ubuntu_path(mda_basename + '_masked.mda')
firings_out = get_ubuntu_path(mda_basename + '_firings.mda')
filt_out_fname = get_ubuntu_path(mda_basename + '_filt.mda')
pre_out_fname = get_ubuntu_path(mda_basename + '_pre.mda')
metrics_out_fname = get_ubuntu_path(mda_basename + '_metrics.json')

raw_fname = get_ubuntu_path(raw_fname)

E:\Apollo_D_Drive\data\VirtualMazeData\b6_august_18_2\NoiseTesting\b6_august_18_2_in_cage_180907_141509_T1_raw.mda


In [19]:
'''raw_fname=None, pre_fname=None, geom_fname=None, params_fname=None,
                 firings_out, filt_out_fname='', pre_out_fname='', metrics_out_fname='', masked_out_fname='',
                 freq_min=300, freq_max=7000, samplerate=30000, detect_sign=1,
                 adjacency_radius=-1, detect_threshold=3, detect_interval=10, clip_size=50,
                 firing_rate_thresh=0.05, isolation_thresh=0.95, noise_overlap_thresh=0.03,
                 peak_snr_thresh=1.5, mask_artifacts='true', mask_threshold=6, mask_chunk_size=2000,
                 mask_num_write_chunks=15'''

"raw_fname=None, pre_fname=None, geom_fname=None, params_fname=None,\n                 firings_out, filt_out_fname='', pre_out_fname='', metrics_out_fname='', masked_out_fname='',\n                 freq_min=300, freq_max=7000, samplerate=30000, detect_sign=1,\n                 adjacency_radius=-1, detect_threshold=3, detect_interval=10, clip_size=50,\n                 firing_rate_thresh=0.05, isolation_thresh=0.95, noise_overlap_thresh=0.03,\n                 peak_snr_thresh=1.5, mask_artifacts='true', mask_threshold=6, mask_chunk_size=2000,\n                 mask_num_write_chunks=15"

In [20]:
# samplerate=int(48e3)
# samplerate=int(24e3)
whiten='true'
samplerate=int(30e3)
detect_interval=20
detect_sign=0
detect_threshold=3
freq_min=300
freq_max=6000
mask_threshold=6
masked_chunk_size = 2000
mask_num_write_chunks=150
clip_size=100

In [15]:
sort_dataset(raw_fname=raw_fname, filt_out_fname=filt_out_fname, pre_out_fname=pre_out_fname, 
             metrics_out_fname=metrics_out_fname, firings_out=firings_out, masked_out_fname=masked_out_fname,
             samplerate=samplerate, detect_interval=detect_interval, detect_sign=detect_sign, 
             detect_threshold=detect_threshold, freq_min=freq_min, freq_max=freq_max, mask_threshold=mask_threshold, 
             mask_chunk_size=masked_chunk_size, mask_num_write_chunks=mask_num_write_chunks, whiten=whiten, clip_size=clip_size
            )

RUNNING: ml-run-process ephys.bandpass_filter --inputs timeseries:/mnt/e/Apollo_D_Drive/data/VirtualMazeData/b6_march_18_1/b6_march_18_1_180403_150327_T1_raw.mda --parameters freq_max:6000 freq_min:300 samplerate:30000 --outputs timeseries_out:/mnt/e/Apollo_D_Drive/data/VirtualMazeData/b6_march_18_1/b6_march_18_1_180403_150327_T1_filt.mda
[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
Process signature: 639a0ac2bafa0256d6c96e087d99aa6da35ad8a5
[ Checking outputs... ]
{"timeseries_out":"/mnt/e/Apollo_D_Drive/data/VirtualMazeData/b6_march_18_1/b6_march_18_1_180403_150327_T1_filt.mda"}
Processing ouput - /mnt/e/Apollo_D_Drive/data/VirtualMazeData/b6_march_18_1/b6_march_18_1_180403_150327_T1_filt.mda
false
{"timeseries_out":"/mnt/e/Apollo_D_Drive/data/VirtualMazeData/b6_march_18_1/b6_march_18_1_180403_150327_T1_filt.mda"}
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Creating links to input files... ]


RUNNING: ml-run-process ms3.cluster_metrics --inputs firings:/mnt/e/Apollo_D_Drive/data/VirtualMazeData/b6_march_18_1/b6_march_18_1_180403_150327_T1_firings.mda timeseries:/mnt/e/Apollo_D_Drive/data/VirtualMazeData/b6_march_18_1/b6_march_18_1_180403_150327_T1_masked.mda --parameters samplerate:30000 --outputs cluster_metrics_out:/mnt/e/Apollo_D_Drive/MountainSortTempJs/mountainlab/tmp_short_term/output_cluster_metrics_out_cdccd82c103604fc103e51cba2df68b1dc5a5560.prv
[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
Process signature: 0ecdb1c6423a619c03820bdb165dd61dbc5f666f
[ Checking outputs... ]
{"cluster_metrics_out":"/mnt/e/Apollo_D_Drive/MountainSortTempJs/mountainlab/tmp_short_term/output_cluster_metrics_out_cdccd82c103604fc103e51cba2df68b1dc5a5560.prv"}
Processing ouput - /mnt/e/Apollo_D_Drive/MountainSortTempJs/mountainlab/tmp_short_term/output_cluster_metrics_out_cdccd82c103604fc103e51cba2df68b1dc5a5560.prv
false
{"

True